In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gemini-2.5-flash', api_key=api_key)

In [2]:
async def name_tool(query: str)->str:
    return "Superman returns"

In [3]:
agent = AssistantAgent(
    name='assistant',
    model_client=model_client,
    tools=[name_tool],
    system_message='Use Tool to get names'
)

In [4]:
result = await agent.run(task='Find information about Superman')
print(result.messages[-1].content)

Superman returns


#### `on_message()` Method

- In AutoGen, agents communicate by sending and receiving messages. The on_message() method is a callback used to define how an agent reacts to a message.
- It intercepts an incoming message and optionally returns a response.
- You can override on_message() to customize the agent's behavior.

#### Cancellation Token

AutoGen agents often run in conversational loops that can continue indefinitely. A cancellation token is a control mechanism to stop or cancel such ongoing interactions.

In [7]:
from autogen_core import CancellationToken

async def assistant_run()-> None:
    response = await agent.on_messages(
        messages= [TextMessage(content='Find information about Superman',source='User')],
        cancellation_token=CancellationToken()
    )

    print(response.inner_messages)
    print('\n\n\n\n')
    print(response.chat_message.content)

await assistant_run()

[ToolCallRequestEvent(id='b32dbd5e-04c8-45ec-b4d1-7cfd9ad4be97', source='assistant', models_usage=RequestUsage(prompt_tokens=139, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 7, 21, 18, 19, 18, 968811, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{"query":"Superman"}', name='name_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='e79af94f-32a4-4c10-843d-56a3b83d140f', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 21, 18, 19, 18, 972037, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Superman returns', name='name_tool', call_id='', is_error=False)], type='ToolCallExecutionEvent')]





Superman returns


#### Streaming Messages

`on_messages_stream()`: method in AutoGen is an advanced hook used to handle streamed message events in a multi-agent chat session. It complements on_message() by allowing an agent to react to messages incrementally as they arrive, instead of waiting for the entire message to be delivered.

| Method                 | When Called                                                                    |
| ---------------------- | ------------------------------------------------------------------------------ |
| `on_message()`         | Called **after** the full message is received.                                 |
| `on_messages_stream()` | Called **as the message is being streamed**, token by token or chunk by chunk. |


In [8]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:

    await Console(
        agent.on_messages_stream(
        messages= [TextMessage(content='Find information about Superman via the tool',source='User')],
        cancellation_token=CancellationToken()
    ),
    output_stats=True # Enable stats Printing
    )


await assistant_run_stream()

---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='', arguments='{"query":"Superman"}', name='name_tool')]
[Prompt tokens: 179, Completion tokens: 15]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='Superman returns', name='name_tool', call_id='', is_error=False)]
---------- assistant ----------
Superman returns
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 179
Total completion tokens: 15
Duration: 2.38 seconds
